# EDA for NCAA March Madness

In [1]:
import os
import glob
import pandas as pd
import re

In [2]:
files = glob.glob('input/MEvents*.csv')

In [3]:
DEBUG = True
if DEBUG:
    NROWS = 50000
else:
    NROWS = None

In [4]:
# Get all csv files started with 'MEvents'
data_frames = [pd.read_csv(file) for file in files]

events = pd.concat(data_frames, axis=0, sort=False)
events.head()

,EventID,Season,DayNum,WTeamID,LTeamID,WFinalScore,LFinalScore,WCurrentScore,LCurrentScore,ElapsedSeconds,EventTeamID,EventPlayerID,EventType,EventSubType,X,Y,Area
0,1,2015,11,1103,1420,74,57,0,0,19,1103,100,miss3,unk,0,0,0
1,2,2015,11,1103,1420,74,57,0,0,19,1420,11784,reb,def,0,0,0
2,3,2015,11,1103,1420,74,57,0,0,27,1420,11789,made2,dunk,0,0,0
3,4,2015,11,1103,1420,74,57,0,0,27,1420,11803,assist,NaN,0,0,0
4,5,2015,11,1103,1420,74,57,0,0,59,1103,87,made2,jump,0,0,0


In [5]:
PATH = 'input/'
dfs = {'MEvents': [], 'Misc': {}}
for root, dirs, files in os.walk(PATH):
    for file in files:
        path_and_file = os.path.join(root, file)
        print(path_and_file)
        if bool(re.search('MEvents', path_and_file)):
            dfs['MEvents'].append(pd.read_csv(path_and_file, nrows=NROWS))
            
        elif bool(re.search('.DS_Store', path_and_file)):
            pass
        else:
            file_name_start_index = path_and_file.rfind('/') + 1
            file_name_end_index = re.search('.csv', path_and_file).span()[0]
            if bool(re.search('MTeamSpellings', path_and_file)):
                dfs['Misc'][path_and_file[file_name_start_index:file_name_end_index]] = pd.read_csv(path_and_file, encoding='cp1252')
            else: 
                dfs['Misc'][path_and_file[file_name_start_index:file_name_end_index]] = pd.read_csv(path_and_file)                
            

input/MEvents2015.csv
input/MEvents2016.csv
input/MEvents2017.csv
input/MEvents2018.csv
input/MEvents2019.csv
input/MPlayers.csv
input/MSampleSubmissionStage1_2020.csv
input/MDataFiles_Stage1\Cities.csv
input/MDataFiles_Stage1\Conferences.csv
input/MDataFiles_Stage1\MConferenceTourneyGames.csv
input/MDataFiles_Stage1\MGameCities.csv
input/MDataFiles_Stage1\MMasseyOrdinals.csv
input/MDataFiles_Stage1\MNCAATourneyCompactResults.csv
input/MDataFiles_Stage1\MNCAATourneyDetailedResults.csv
input/MDataFiles_Stage1\MNCAATourneySeedRoundSlots.csv
input/MDataFiles_Stage1\MNCAATourneySeeds.csv
input/MDataFiles_Stage1\MNCAATourneySlots.csv
input/MDataFiles_Stage1\MRegularSeasonCompactResults.csv
input/MDataFiles_Stage1\MRegularSeasonDetailedResults.csv
input/MDataFiles_Stage1\MSeasons.csv
input/MDataFiles_Stage1\MSecondaryTourneyCompactResults.csv
input/MDataFiles_Stage1\MSecondaryTourneyTeams.csv
input/MDataFiles_Stage1\MTeamCoaches.csv
input/MDataFiles_Stage1\MTeamConferences.csv
input/MDataFil

In [6]:
# Concatenate MEvents2015, 2016, 2017, 2018, 2019 together
events = pd.concat(dfs['MEvents'], axis=0, sort=False)
events.head()

,EventID,Season,DayNum,WTeamID,LTeamID,WFinalScore,LFinalScore,WCurrentScore,LCurrentScore,ElapsedSeconds,EventTeamID,EventPlayerID,EventType,EventSubType,X,Y,Area
0,1,2015,11,1103,1420,74,57,0,0,19,1103,100,miss3,unk,0,0,0
1,2,2015,11,1103,1420,74,57,0,0,19,1420,11784,reb,def,0,0,0
2,3,2015,11,1103,1420,74,57,0,0,27,1420,11789,made2,dunk,0,0,0
3,4,2015,11,1103,1420,74,57,0,0,27,1420,11803,assist,NaN,0,0,0
4,5,2015,11,1103,1420,74,57,0,0,59,1103,87,made2,jump,0,0,0


In [7]:
# Merge events and MPlayers
events = events.merge(dfs['Misc']['MPlayers'],
             how='left',
             left_on = ['EventTeamID', 'EventPlayerID'],
             right_on = ['TeamID', 'PlayerID'])

In [8]:
events.drop(['PlayerID', 'TeamID'], axis=1, inplace=True)

In [9]:
events.head()

,EventID,Season,DayNum,WTeamID,LTeamID,WFinalScore,LFinalScore,WCurrentScore,LCurrentScore,ElapsedSeconds,EventTeamID,EventPlayerID,EventType,EventSubType,X,Y,Area,LastName,FirstName
0,1,2015,11,1103,1420,74,57,0,0,19,1103,100,miss3,unk,0,0,0,McAdams,Reggie
1,2,2015,11,1103,1420,74,57,0,0,19,1420,11784,reb,def,0,0,0,Grant,Jourdan
2,3,2015,11,1103,1420,74,57,0,0,27,1420,11789,made2,dunk,0,0,0,Joyce,Cody
3,4,2015,11,1103,1420,74,57,0,0,27,1420,11803,assist,NaN,0,0,0,Sparow,Wayne
4,5,2015,11,1103,1420,74,57,0,0,59,1103,87,made2,jump,0,0,0,Forsythe,Pat


LastName and FirstName are appended in events dataFrame

## Very Simple Feature Engineering

#### Make Full name using LastNme and FirstName

In [18]:
events['FullName'] = events['FirstName'] + ' ' + events['LastName']

#### Make Quarter using ElapsedSeconds

In [27]:
# Max ElapsedSeconds
events.ElapsedSeconds.max()

3600

The max playing time is 3600, which means this game had four overtime play.

In [ ]:
quarter_bins = [0, 2400, 2700, 3000, 3300, 3600]
quarter_labels = [1, 2, 3, 4, 5, 6]
events['Overtime'] = pd.cut(events['ElapsedSeconds'], 
                           bins=quarter_bins, labels=quarter_labels)

- 1 means 1st half
- 2 means 2nd half
- 3 means 1st overtime
- 4 means 2nd overtime
- 5 means 3rd overtime
- 6 means 4th overtime

## EDA

 - Event type별로 코트 위에 plot 그리기
 - 스코어 차이 bar plot
 